In [1]:
import numpy as np
from numpy import nan

from tabulate import tabulate
import random
from itertools import zip_longest
from collections import namedtuple
from stack import Stack
from actions import Moove, Action
from utils_class import EpsilonGreedy, ReplayMemory, Agent
from DQN import DQNAgent    

%load_ext autoreload
%autoreload 2


2022-05-24 09:31:18.552515: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 09:31:18.552552: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
EPISODE = 2500
DECAY = 0.998
MAX_EPSILON = 0.99
MIN_EPSILON = 0.15
ACTION_REWARD = -1
FINISH_REWARD = 100
LEARNING_RATE = 0.1
DISCOUNT = 0.95
STACK_SIZE = 10
CAPACITY = 100
BATCH_SIZE = 64
MIN_REPLAY_MEMORY_SIZE = 100
GAMMA = 0.99


In [8]:
Experience = namedtuple(
                'Experience', ('state', 'action', 'next_state', 'reward'))
class Env:
    
    
    def __init__(self, size):
        self.A = Stack(size, 0)
        self.B = Stack(size, 1)
        self.size = size
        self.state_t = np.vstack((self.A.stack, self.B.stack))
        self.state_t1 = np.vstack((self.A.stack, self.B.stack))
        
        self.moover = Moove(self.A, self.B, size)
        self.actions = Action(self.A, self.B)
        self.current_action = 1;
        # self.agent = Agent(EpsilonGreedy(MIN_EPSILON, MAX_EPSILON, DECAY), 11)
        self.strategy = EpsilonGreedy(MIN_EPSILON, MAX_EPSILON, DECAY)
        self.agent = DQNAgent(size)
        
        self.cum_reward = 0
        self.step = 0
        
        self.experience = None
        self.replaymemory = ReplayMemory(CAPACITY)
    
    def reset(self, size):
        self.A = Stack(size)
        self.B = Stack(0)
        
    def actions_available(self):
        return self.actions.possible_actions()
    
    def reward(self):
        if self.B.stack.size == 0 and np.all(self.A.stack[:-1] <= self.A.stack[1:]):
             self.cum_reward += 100
        self.cum_reward += self.moover.reward
        return (self.cum_reward)
        
    def choose_action(self):
        exploration_rate = self.strategy.get_exploration_rate(self.step)
        
        self.step += 1
        
        if (exploration_rate > random.random()):
            #exploration
            self.current_action = np.random.choice(self.actions.possible_actions())
            return self.current_action
        else :
            self.current_action = np.argmax(self.agent.policy_model.predict(self.state))
            return self.current_action
    
    def take_actions(self):
        if self.current_action == 0:
            self.cum_reward += self.moover.push_b(self.A, self.B)
            
        elif self.current_action == 1:
            self.cum_reward += self.moover.push_a(self.A, self.B)
            
        elif self.current_action == 2:
            self.cum_reward += self.moover.rotate_b(self.B)
            
        elif self.current_action == 3:
            self.cum_reward += self.moover.rotate_a(self.A)
            
        elif self.current_action == 4:
            self.cum_reward += self.moover.inverse_rotate_b(self.B)
            
        elif self.current_action == 5:
            self.cum_reward += self.moover.inverse_rotate_a(self.A)
            
        elif self.current_action == 6:
           self.cum_reward += self.moover.swap_b(self.B)
            
        elif self.current_action == 7:
            self.cum_reward += self.moover.swap_a(self.A)
            
        elif self.current_action == 8:
           self.cum_reward += self.moover.rotate(self.A, self.B)
            
        elif self.current_action == 9:
            self.cum_reward += self.moover.reverse(self.A, self.B)
            
        elif self.current_action == 10:
            self.cum_reward += self.moover.swap(self.A, self.B)
            
        else :
            pass
        # self.state_t1 = np.vstack((self.A.stack, self.B.stack))
    
    def state(self):
        return np.vstack((self.A.stack, self.B.stack))

    def action_space(self):
      return np.array([0,
                       1,
                       2,
                       3,
                       4,
                       5,
                       6,
                       7,
                       8,
                       9,
                       10])
      
    
    def create_experience(self):
        self.experience = Experience(self.state_t, self.current_action, self.cum_reward, self.state_t1)
    
    def print_stack(self):
        print("-------------------")
        for a, b in zip_longest(reversed(self.A.stack), reversed(self.B.stack), fillvalue='xxx'):
          
          print(f"{a}\t{b}")
        print("-------------------")
        # print(f"-----Reward {self.reward()}")
  


In [9]:
size = 10
env = Env(size)

2022-05-24 09:32:47.170804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 09:32:47.171164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 09:32:47.171201: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-24 09:32:47.171224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-24 09:32:47.171248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [10]:
env.print_stack()

# env.moover.push_b()
# env.choose_action()
env.current_action = 0
env.take_actions()
print(f"Reward -> {env.cum_reward}")
env.take_actions()
print(f"Reward -> {env.cum_reward}")
env.take_actions()
print(f"Reward -> {env.cum_reward}")
env.take_actions()
print(f"Reward -> {env.cum_reward}")
# env.current_action = 1
# env.take_actions()
# env.current_action = 2
# env.take_actions()
# env.current_action = 3
# env.take_actions()
# env.current_action = 4
# env.take_actions()
# env.current_action = 5
# env.take_actions()
# env.current_action = 6
# env.take_actions()
# env.current_action = 7
# env.take_actions()
# env.current_action = 8
# env.take_actions()
# env.current_action = 9
# env.take_actions()
# env.current_action = 10
# env.take_actions()


env.print_stack()



-------------------
27.0	nan
32.0	nan
43.0	nan
87.0	nan
36.0	nan
6.0	nan
97.0	nan
67.0	nan
5.0	nan
40.0	nan
-------------------
Reward -> -1
Reward -> -2
Reward -> -3
Reward -> -4
-------------------
36.0	87.0
6.0	43.0
97.0	32.0
67.0	27.0
5.0	nan
40.0	nan
nan	nan
nan	nan
nan	nan
nan	nan
-------------------


In [11]:
env.current_action = 8
env.take_actions()
print(f"Reward -> {env.cum_reward}")
env.print_stack()


Reward -> -6
-------------------
40.0	27.0
36.0	87.0
6.0	43.0
97.0	32.0
67.0	nan
5.0	nan
nan	nan
nan	nan
nan	nan
nan	nan
-------------------


In [ ]:

env.print_stack()  
print(f"Reward -> {env.reward()}") 

IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
env.moover.inverse_rotate_b()
env.actions.balanced(size)
env.print_stack()   

-------------------
30.0	61.0
89.0	32.0
66.0	70.0
7.0	91.0
nan	2.0
nan	10.0
nan	nan
nan	nan
nan	nan
nan	nan
-------------------
